# MedGemma-Council: Multi-Agent Clinical Decision Support

This notebook demonstrates the MedGemma-Council system — a "Council of Experts" 
that debates clinical cases via a LangGraph state machine, powered by MedGemma 1.5 models.

## Architecture
```
Ingestion -> Supervisor Route -> Specialists -> Conflict Check
  -> [conflict] -> Research (PubMed) -> Debate -> Conflict Check (loop)
  -> [no conflict] -> Synthesis -> Final Plan
```

## Available Specialists
- **Cardiology** (ACC/AHA), **Oncology** (NCCN), **Pediatrics** (AAP/WHO)
- **Radiology** (vision-based), **Psychiatry** (APA/DSM-5-TR)
- **Emergency Medicine** (ACLS/ATLS), **Dermatology** (AAD)
- **Research** (PubMed/MEDLINE literature retrieval)

## 1. Installation

In [ ]:
# Install the medgemma-council package
# On Kaggle, the repo should be uploaded as a dataset or utility script
import os

# If running on Kaggle:
REPO_PATH = "/kaggle/working/medgemma-council"
if not os.path.exists(REPO_PATH):
    # Fallback for local development
    REPO_PATH = "."

# Install dependencies
!pip install -q langgraph langchain-core pydantic biopython

# Install the package itself
!pip install -q -e {REPO_PATH}

In [ ]:
# Verify installation
import sys
sys.path.insert(0, os.path.join(REPO_PATH, "src"))

from graph import CouncilState, build_council_graph
from utils.safety import scan_for_red_flags, redact_pii
print("Installation verified successfully!")

## 2. Run Tests (Verify Integrity)

In [ ]:
# Run the test suite to verify everything works
# All 146+ tests should pass in < 1 second (everything is mocked)
!cd {REPO_PATH} && python -m pytest tests/ -v --tb=short 2>&1 | tail -20

## 3. CLI-First Workflow (Recommended for Kaggle)

The `council_cli` module provides the simplest way to run the council 
programmatically without any web framework.

In [ ]:
# Import the CLI module
sys.path.insert(0, REPO_PATH)
from council_cli import run_council_cli, format_result, build_state

### Example 1: Cardiology Case

In [ ]:
# Run a cardiology case
result = run_council_cli(
    age=65,
    sex="Male",
    chief_complaint="Chest pain radiating to left arm, onset 2 hours ago",
    history="Hypertension, Type 2 Diabetes, former smoker (quit 5 years ago)",
    medications=["Metformin 1000mg BID", "Lisinopril 20mg daily", "Aspirin 81mg daily"],
    vitals={"bp": "160/95", "hr": 92, "temp": 98.6, "spo2": 96, "rr": 18},
    labs={"troponin": 0.08, "bnp": 450, "creatinine": 1.4, "glucose": 210},
)

# Display formatted results
print(format_result(result, output_format="text"))

### Example 2: Pediatrics Case

In [ ]:
result_peds = run_council_cli(
    age=5,
    sex="Female",
    chief_complaint="High fever (103F) for 3 days with cough and poor appetite",
    history="No significant past medical history, vaccinations up to date",
    medications=[],
    vitals={"temp": 103.0, "hr": 130, "rr": 28, "spo2": 94},
    labs={"wbc": 18000, "crp": 45},
)

print(format_result(result_peds, output_format="text"))

### Example 3: Multi-specialty Case (Oncology + Cardiology)

In [ ]:
result_multi = run_council_cli(
    age=58,
    sex="Female",
    chief_complaint="Newly diagnosed breast cancer with pre-existing heart failure",
    history="HFrEF (LVEF 35%), NYHA Class II, breast mass found on screening mammography",
    medications=["Carvedilol 25mg BID", "Sacubitril/Valsartan 97/103mg BID", "Spironolactone 25mg"],
    vitals={"bp": "110/70", "hr": 68, "spo2": 97},
    labs={"bnp": 890, "troponin": 0.02, "lvef": 35},
)

print(format_result(result_multi, output_format="text"))

## 4. Working with the Raw State

For more control, you can work directly with the CouncilState.

In [ ]:
# Build state manually
state = build_state(
    age=34,
    sex="Male",
    chief_complaint="Severe anxiety, insomnia for 3 months, suicidal ideation",
    history="Major depressive disorder, generalized anxiety disorder",
    medications=["Sertraline 100mg daily"],
)

# Inspect the state structure
print("State keys:", list(state.keys()))
print("Patient context:", state["patient_context"])

In [ ]:
# Check for red flags BEFORE running the council
from utils.safety import scan_for_red_flags

flags = scan_for_red_flags(state["patient_context"]["chief_complaint"])
print("Red flags detected:", flags["flagged"])
if flags["flagged"]:
    print("Flags:", flags["flags"])
    print("Emergency message:", flags["emergency_message"])

## 5. JSON Output (for downstream processing)

In [ ]:
import json

# Get JSON output for programmatic use
json_output = format_result(result, output_format="json")
parsed = json.loads(json_output)

print("Final Plan:", parsed["final_plan"][:200], "...")
print("Consensus:", parsed["consensus_reached"])
print("Specialists consulted:", list(parsed["agent_outputs"].keys()))

## 6. Using Medical Images

The RadiologyAgent and DermatologyAgent can process medical images 
using the MedGemma 1.5 4B multimodal model.

In [ ]:
# Example with images (paths should point to actual image files)
# On Kaggle, images would be in /kaggle/input/your-dataset/

# result_with_images = run_council_cli(
#     age=72,
#     sex="Male",
#     chief_complaint="Persistent cough, weight loss, hemoptysis",
#     history="40 pack-year smoking history",
#     image_paths=[
#         "/kaggle/input/chest-xrays/current_cxr.png",
#         "/kaggle/input/chest-xrays/prior_cxr_6mo.png",
#     ],
# )
# print(format_result(result_with_images))

print("Image processing requires MedGemma 1.5 4B model on GPU.")
print("Uncomment the code above when running on Kaggle with T4 GPUs.")

## 7. Model Loading (GPU Setup)

On Kaggle with T4 GPUs, load the actual MedGemma models:

In [ ]:
# GPU Model Loading (uncomment on Kaggle with T4 GPUs)
#
# from utils.model_loader import ModelLoader
#
# loader = ModelLoader()
#
# # Load MedGemma-27B quantized text model on GPU 0
# text_model = loader.load_text_model(
#     name="medgemma-27b",
#     model_path="/kaggle/input/medgemma/medgemma-27b-Q4_K_M.gguf",
#     n_gpu_layers=40,
#     n_ctx=4096,
# )
#
# # Load MedGemma 1.5 4B vision model on GPU 1
# vision_model = loader.load_vision_model(
#     name="medgemma-4b",
#     model_id="google/medgemma-4b-it",
#     device_map="cuda:1",
#     torch_dtype="float16",
# )
#
# print(f"Loaded models: {list(loader.loaded_models.keys())}")

print("Model loading requires Kaggle T4 GPU environment.")
print("See utils/model_loader.py for VRAM management details.")

## 8. Gradio UI (Alternative to Streamlit)

For interactive use on Kaggle (where Streamlit doesn't work well), 
use the Gradio interface:

In [ ]:
# Install Gradio if not already installed
# !pip install -q gradio

# Launch the Gradio interface
# This will create an interactive UI in the notebook output
#
# import subprocess
# subprocess.Popen(["python", os.path.join(REPO_PATH, "app_gradio.py")])

print("To launch Gradio UI:")
print(f"  python {os.path.join(REPO_PATH, 'app_gradio.py')}")
print("Or import and launch directly:")
print("  from app_gradio import create_interface")
print("  demo = create_interface()")
print("  demo.launch(share=True)  # share=True for Kaggle")

## 9. Safety Guardrails Demo

In [ ]:
from utils.safety import scan_for_red_flags, redact_pii, add_disclaimer

# Red flag detection
test_cases = [
    "Patient denies suicidal ideation but reports self-harm behavior.",
    "Lactate > 4, MAP < 65, suspected septic shock.",
    "Patient stable, vitals within normal limits.",
    "Acute ischemic stroke, onset 45 minutes ago.",
]

for text in test_cases:
    result = scan_for_red_flags(text)
    status = "RED FLAG" if result["flagged"] else "CLEAR"
    flags = ", ".join(result["flags"]) if result["flags"] else "none"
    print(f"[{status}] {text[:60]}... -> {flags}")

In [ ]:
# PII redaction
text_with_pii = (
    "Patient John Smith, SSN 123-45-6789, phone (555) 123-4567, "
    "email john@hospital.com, MRN: 12345678, presents with chest pain."
)

print("Before redaction:")
print(text_with_pii)
print("\nAfter redaction:")
print(redact_pii(text_with_pii))

---

## Summary

This notebook demonstrated:

1. **Installation** — `pip install -e .` for the medgemma-council package
2. **CLI workflow** — `run_council_cli()` for programmatic use
3. **Multiple case types** — Cardiology, Pediatrics, Multi-specialty
4. **Safety guardrails** — Red flag detection, PII redaction, disclaimers
5. **Output formats** — Text (human-readable) and JSON (machine-readable)
6. **GPU setup** — Model loading with VRAM management
7. **Gradio UI** — Interactive web interface for Kaggle

For production use on Kaggle T4 GPUs, uncomment the model loading cells 
and ensure the MedGemma model weights are available as a dataset.